In [3]:
import pandas as pd

wine=pd.read_csv("https://bit.ly/wine_csv_data")

In [5]:
data=wine[ ['alcohol', 'sugar', 'pH'] ].to_numpy()
target=wine['class'].to_numpy()

In [6]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(data,target,test_size=0.2,random_state=42)

In [8]:
# 훈련세트에서 검증세트 추출
#sub_x,sub_y=훈련세트
#val_x,val_y=검증세트

sub_x,val_x,sub_y,val_y=train_test_split(X_train,y_train,test_size=0.2,random_state=42)

In [9]:
sub_x.shape,val_x.shape

((4157, 3), (1040, 3))

In [10]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=42)
dt.fit(sub_x,sub_y)
print(dt.score(sub_x,sub_y))
print(dt.score(val_x,val_y))

0.9971133028626413
0.864423076923077


In [11]:
from sklearn.model_selection import cross_validate

scores=cross_validate(dt,X_train,y_train)
print(scores)

{'fit_time': array([0.00598598, 0.00598478, 0.00598383, 0.00596857, 0.00698233]), 'score_time': array([0.0009954 , 0.00099611, 0.00101304, 0.00099707, 0.00099921]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [13]:
#검증폴드의 점수
#테스트세트의 점수가 아님

import numpy as np
np.mean(scores['test_score'])

0.855300214703487

In [15]:
#교차검증에서 훈련세트를 섞으려면 분할기를 사용해야함 (교차검증이 아닐시에는 train_test_split에서 자동으로 훈련세트가 섞인다.)
# 분할기의 종류 : 회귀-KFold , 분류-StartfieldKFold

from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

scores=cross_validate(dt,X_train,y_train,cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [16]:
# 10-Fold
#method-1
spliter=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
score=cross_validate(dt,X_train,y_train,cv=spliter)

print(np.mean(score['test_score']))

0.8574181117533719


In [17]:
#method-2
score=cross_validate(dt,X_train,y_train,cv=StratifiedKFold(n_splits=10,shuffle=True,random_state=42))

print(np.mean(score['test_score']))

0.8574181117533719
